In [1]:
from autocat.surface import generate_surface_structures
from autocat.saa import generate_saa_structures

from autocat.utils import extract_structures

from autocat.learning.featurizers import Featurizer

from dscribe.descriptors import CoulombMatrix
from matminer.featurizers.composition import ElementProperty

In this example we show how to use `AutoCat` to featurize structures with the `Featurizer` class.

Here we will be featurizing mono-elemental surfaces.

In [2]:
# Generate structures to be featurized
mono_surfaces_dictionary = generate_surface_structures(
    species_list=["Fe", "Ru", "Cu", "Pd"],
    facets={"Fe": ["110"], "Ru":["0001"], "Cu":["111"], "Pd":["111"]}
)
mono_surfaces_structures = extract_structures(mono_surfaces_dictionary)

saa_surfaces_dictionary = generate_saa_structures(
    host_species=["Cu", "Au"],
    dopant_species=["Pt", "Pd"],
    facets={"Cu":["111"], "Au":["111"]}
)
saa_surfaces_structures = extract_structures(saa_surfaces_dictionary)

all_structures = mono_surfaces_structures.copy()
all_structures.extend(saa_surfaces_structures)

In [3]:
print(all_structures[0].get_chemical_formula())

Fe36


In [4]:
# Instantiate featurizer based on Coulomb Matrix
coulomb_featurizer = Featurizer(
    featurizer_class=CoulombMatrix, 
    design_space_structures=all_structures
)
print(coulomb_featurizer)

+-----------------------------------+-------------------------------------------------+
|                                   |                    Featurizer                   |
+-----------------------------------+-------------------------------------------------+
|               class               | dscribe.descriptors.coulombmatrix.CoulombMatrix |
|               kwargs              |                       None                      |
|            species list           |       ['Fe', 'Ru', 'Pt', 'Pd', 'Au', 'Cu']      |
|       maximum structure size      |                        36                       |
|               preset              |                       None                      |
| design space structures provided? |                       True                      |
+-----------------------------------+-------------------------------------------------+


In [5]:
# Featurize just Fe
fe_feature_vector = coulomb_featurizer.featurize_single(all_structures[0])
print(fe_feature_vector.shape)

(1296,)


In [6]:
# Featurize all structures into a single matrix
feature_matrix = coulomb_featurizer.featurize_multiple(all_structures)
print(feature_matrix.shape)

(8, 1296)


In [7]:
# Instantiate element property featurizer
element_featurizer = Featurizer(
    featurizer_class=ElementProperty,
    design_space_structures=all_structures,
    preset="matminer"
)

print(element_featurizer)

+-----------------------------------+------------------------------------------------------------+
|                                   |                         Featurizer                         |
+-----------------------------------+------------------------------------------------------------+
|               class               | matminer.featurizers.composition.composite.ElementProperty |
|               kwargs              |                            None                            |
|            species list           |            ['Fe', 'Ru', 'Pt', 'Pd', 'Au', 'Cu']            |
|       maximum structure size      |                             36                             |
|               preset              |                          matminer                          |
| design space structures provided? |                            True                            |
+-----------------------------------+------------------------------------------------------------+


In [8]:
# Featurize just Fe
fe_feature_vector = element_featurizer.featurize_single(all_structures[0])
print(fe_feature_vector.shape)

(65,)


In [9]:
# Featurize all structures at once
feature_matrix = element_featurizer.featurize_multiple(all_structures)
print(feature_matrix.shape)

(8, 65)
